In [86]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Cleaning the data

In [31]:
df = pd.read_csv('../data/external/COVID-19_Case_Surveillance_Public_Use_Data.csv', low_memory = False)
# drop dates
df = df.drop(['cdc_report_dt', 'pos_spec_dt', 'onset_dt'], axis = 1)
# use only confirmed cases
df = df[df['current_status'] == 'Laboratory-confirmed case']
# use only males and females
df = df[(df['sex']=='Male') | (df['sex']=='Female')]
# use only known ages
index_names = df[df['age_group'] == 'Unknown'].index 
df = df.drop(index_names)
# use only known ethnicities
index_names = df[df['Race and ethnicity (combined)'] == 'Unknown'].index 
df = df.drop(index_names)
# use only known hospitalization status
df = df[(df['hosp_yn']=='Yes') | (df['hosp_yn']=='No')]
# remove unknown icu status
df = df[(df['icu_yn']=='Yes') | (df['icu_yn']=='No')]
# removing unknown death status
df = df[(df['death_yn']=='Yes') | (df['death_yn']=='No')]
df = df[(df['medcond_yn']=='Yes') | (df['medcond_yn']=='No')]
df.shape

(328977, 8)

## Saving the data

In [32]:
df.to_csv('../data/processed/clean.csv', index = False)

In [51]:
data = pd.read_csv('../data/processed/clean.csv', low_memory = False)
deathsdf = pd.read_csv('../data/processed/Y.csv')
df = pd.read_csv('../data/processed/X.csv')
data.sample(10)

,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,medcond_yn
102826,Laboratory-confirmed case,Female,30 - 39 Years,"White, Non-Hispanic",No,No,No,No
158274,Laboratory-confirmed case,Male,40 - 49 Years,"White, Non-Hispanic",No,No,No,No
210513,Laboratory-confirmed case,Male,60 - 69 Years,"White, Non-Hispanic",Yes,No,No,Yes
202619,Laboratory-confirmed case,Male,50 - 59 Years,"White, Non-Hispanic",No,No,No,No
268731,Laboratory-confirmed case,Female,20 - 29 Years,"White, Non-Hispanic",No,No,No,No
151767,Laboratory-confirmed case,Female,30 - 39 Years,"White, Non-Hispanic",No,No,No,Yes
302526,Laboratory-confirmed case,Female,30 - 39 Years,"White, Non-Hispanic",No,No,No,Yes
252129,Laboratory-confirmed case,Male,60 - 69 Years,"Black, Non-Hispanic",Yes,No,No,Yes
188493,Laboratory-confirmed case,Female,40 - 49 Years,Hispanic/Latino,Yes,No,No,Yes
274350,Laboratory-confirmed case,Female,20 - 29 Years,"White, Non-Hispanic",Yes,No,No,Yes


## Linear Model

In [87]:
lr = linear_model.LinearRegression()
predicted = lr.fit(X=df, y=deathsdf['death_yn'])

In [79]:
print(results.coef_)

[ 0.00000000e+00  1.72535273e+11  1.72535273e+11 -3.72078355e+10
 -3.72078355e+10 -3.72078355e+10 -3.72078355e+10 -3.72078355e+10
 -3.72078355e+10 -3.72078355e+10 -3.72078355e+10 -3.72078355e+10
 -5.93242778e+10 -5.93242778e+10 -5.93242778e+10 -5.93242778e+10
 -5.93242778e+10 -5.93242778e+10 -5.93242778e+10 -2.33055962e+11
 -2.33055962e+11  1.14864817e+12  1.14864817e+12  1.89421626e+12
  1.89421626e+12]


In [80]:
values = np.append(predicted.intercept_, results.coef_)
newResults = pd.DataFrame(values, index = names, columns=['Coefficient'])
print(newResults.sort_values('Coefficient', ascending=False).head(10))

                                           Coefficient
medcond_yn_Yes                            1.894216e+12
medcond_yn_No                             1.894216e+12
icu_yn_Yes                                1.148648e+12
icu_yn_No                                 1.148648e+12
sex_Male                                  1.725353e+11
sex_Female                                1.725353e+11
current_status_Laboratory-confirmed case  0.000000e+00
age_group_80+ Years                      -3.720784e+10
age_group_70 - 79 Years                  -3.720784e+10
age_group_60 - 69 Years                  -3.720784e+10


In [95]:
yPredicted = predicted.predict(deathsdf['death_yn'])
plt.scatter(df, yPredicted.values.reshape(-1, 1), color='blue', linewidth=3)

ValueError: Expected 2D array, got 1D array instead:
array=[0 0 0 ... 0 0 1].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.